In [4]:
import time
import os
from bs4 import BeautifulSoup as BS
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.options import Options
import pandas as pd
import numpy as np
import datetime
from datetime import datetime, timedelta

def user_pick(options):
    print("Please choose from below by number:")
    for idx, element in enumerate(options):
        print("{}) {}".format(idx + 1, element))
    i = input("Enter number: ")
    try:
        if 0 < int(i) <= len(options):
            return int(i) - 1
    except:
        pass
    return int(i) - 1

In [2]:
sportoptions = ['football','hockey','american-football']
sportnum = user_pick(sportoptions)
print("you selected: " + sportoptions[sportnum])
sport = sportoptions[sportnum]

countryoptions = ['england','sweden','italy','france','germany','usa','spain']
countrynum = user_pick(countryoptions)
print("you selected: " + countryoptions[countrynum])
country = countryoptions[countrynum]

options = Options()
options.add_argument('log-level=1')
driver = webdriver.Chrome(r"G:\Shared drives\UK Data\Data Team - Internal\MattH's Folder\Python Scripts\ChromeDriver\chromedriver.exe", options=options)
driver.maximize_window()
# Go to flashscore website
driver.get('https://www.flashscore.com/');
WebDriverWait(driver, 200).until(EC.presence_of_element_located((By.XPATH,"//*[@id='onetrust-accept-btn-handler']")))
driver.find_element(By.XPATH,"//*[@id='onetrust-accept-btn-handler']").click()
time.sleep(1)
new_url = 'https://www.flashscore.com/'+ sport +'/' + country
eventlist = []
driver.get(new_url)
WebDriverWait(driver, 200).until(EC.presence_of_element_located((By.XPATH, "//*[@id='mt']/div/div[2]")))
try:
    driver.find_element(By.XPATH,"//*[@id='onetrust-accept-btn-handler']").click()
except:
    pass 
try:
    driver.find_element(By.CSS_SELECTOR,'div.show-more.leftMenu__item.leftMenu__item--more').click()
except:
    pass
Page1 = BS(driver.page_source, 'html.parser')
Comps_tbl = Page1.find("div", {"class":"menu selected-country-list leftMenu leftMenu--selected"})
Comps = Comps_tbl.find_all("div", {"class":"leftMenu__item leftMenu__item--width"})
CompList = []
for row in Comps:
    Comp = row.get_text()
    Link = "https://www.flashscore.com" + row.find("a")['href']
    CompList.append([Comp,Link])
CList = pd.DataFrame(CompList, columns = ['Competion','Link'])
comptionlist = CList['Competion']
compnum = user_pick(comptionlist)
print("you selected: " + comptionlist[compnum])
driver.get(CList.Link[compnum] +"archive/")

Page2 = BS(driver.page_source, 'html.parser')
Season_tbl = Page2.find("section", {"class":"archive"})
Seasons = Season_tbl.find_all("div", {"class":"archive__season"})
SeasonList = []
for row in Seasons:
    if row.get_text().strip() != 'Season':
        Season = row.get_text().strip()
        Link = "https://www.flashscore.com" + row.find("a")['href']
        SeasonList.append([Season,Link])
SList = pd.DataFrame(SeasonList, columns = ['Season','Link'])
seaslist = SList['Season']
seasonnum = user_pick(seaslist)
print("you selected: " + seaslist[seasonnum])    
driver.get(SList.Link[seasonnum] +"results/")

while True:
    try:
        time.sleep(1)
        driver.find_element(By.CSS_SELECTOR,'#live-table > div.event.event--results > div > div > a').click()
    except:
        break
FinalData = []
CompetionName = CList.Competion[compnum]
SeasonName = SList.Season[seasonnum]
ResultPage = BS(driver.page_source, 'html.parser')
Year = ResultPage.find("div", {"class":"heading__info"}).get_text()
ResultTable = ResultPage.find("div", {"class":"event event--results"})
label = ResultTable.find("span", {"class":"event__title--name"}).get_text()
Rows = ResultTable.find_all("div",{"class":["event__round event__round--static",
                                            "event__match event__match--static event__match--twoLine",
                                            "event__match event__match--static event__match--last event__match--twoLine"]})
for row in Rows:
    if row['class'] == ['event__round', 'event__round--static']:
        matchday = row.get_text()
    elif row['class'] == ['event__match', 'event__match--static', 'event__match--last', 'event__match--twoLine']:
        dt = row.find("div",{"class":"event__time"}).get_text()
        home = row.find("div",{"class":["event__participant event__participant--home fontExtraBold","event__participant event__participant--home"]}).get_text()
        away = row.find("div",{"class":["event__participant event__participant--away fontExtraBold","event__participant event__participant--away"]}).get_text()
        mdatetime = dt[0:6] + Year + dt[6:12]
        mdate = mdatetime[0:10]
        mtime = mdatetime[-5:]
        FinalData.append([CompetionName,SeasonName,matchday,mdate,mtime,home,away])          
    elif row['class'] == ['event__match', 'event__match--static', 'event__match--twoLine']:
        dt = row.find("div",{"class":"event__time"}).get_text()
        home = row.find("div",{"class":["event__participant event__participant--home fontExtraBold","event__participant event__participant--home"]}).get_text()
        away = row.find("div",{"class":["event__participant event__participant--away fontExtraBold","event__participant event__participant--away"]}).get_text()
        mdatetime = dt[0:6] + Year + dt[6:12]
        mdate = mdatetime[0:10]
        mtime = mdatetime[-5:]
        FinalData.append([CompetionName,SeasonName,matchday,mdate,mtime,home,away])
    elif row['class'] == ['event__match', 'event__match--static', 'event__match--doubles', 'event__match--twoLine']:
        dt = row.find("div",{"class":"event__time"}).get_text()
        home = row.find("div",{"class":["event__participant event__participant--doubles event__participant--home fontExtraBold event__participant--home1","event__participant event__participant--doubles event__participant--home event__participant--home1"]}).get_text()
        away = row.find("div",{"class":["event__participant event__participant--doubles event__participant--away fontExtraBold event__participant--away1","event__participant event__participant--doubles event__participant--away event__participant--away1"]}).get_text()
        mdatetime = dt[0:6] + Year + dt[6:12]
        mdate = mdatetime[0:10]
        mtime = mdatetime[-5:]
        FinalData.append([CompetionName,SeasonName,matchday,mdate,mtime,home,away])
    elif row['class'] == ['event__match', 'event__match--static', 'event__match--last', 'event__match--doubles', 'event__match--twoLine']:
        dt = row.find("div",{"class":"event__time"}).get_text()
        home = row.find("div",{"class":["event__participant event__participant--doubles event__participant--home fontExtraBold event__participant--home1","event__participant event__participant--doubles event__participant--home event__participant--home1"]}).get_text() + " / " + row.find("div",{"class":["event__participant event__participant--doubles event__participant--home fontExtraBold event__participant--home2","event__participant event__participant--doubles event__participant--home event__participant--home2"]}).get_text()
        away = row.find("div",{"class":["event__participant event__participant--doubles event__participant--away fontExtraBold event__participant--away1","event__participant event__participant--doubles event__participant--away event__participant--away1"]}).get_text() + " / " + row.find("div",{"class":["event__participant event__participant--doubles event__participant--away fontExtraBold event__participant--away2","event__participant event__participant--doubles event__participant--away event__participant--away2"]}).get_text()
        mdatetime = dt[0:6] + Year + dt[6:12]
        mdate = mdatetime[0:10]
        mtime = mdatetime[-5:]
        FinalData.append([CompetionName,SeasonName,matchday,mdate,mtime,home,away])
df = pd.DataFrame(FinalData, columns = ['Competion','Season','matchday','date','time','home','away'])
print(str(len(FinalData)) + " Matches Recorded")

Please choose from below by number:
1) football
2) hockey
3) american-football
Enter number: 1
you selected: football
Please choose from below by number:
1) england
2) sweden
3) italy
4) france
5) germany
6) usa
7) spain
Enter number: 7
you selected: spain


<ipython-input-2-f9490e37e73c>:13: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(r"G:\Shared drives\UK Data\Data Team - Internal\MattH's Folder\Python Scripts\ChromeDriver\chromedriver.exe", options=options)


Please choose from below by number:
1) LaLiga
2) LaLiga2
3) Primera RFEF - Group 1
4) Primera RFEF - Group 2
5) Primera RFEF - Play Offs
6) Primera RFEF - Promotion Play Offs
7) Segunda RFEF - Group 1
8) Segunda RFEF - Group 2
9) Segunda RFEF - Group 3
10) Segunda RFEF - Group 4
11) Segunda RFEF - Group 5
12) Segunda RFEF - Promotion Play Offs
13) Segunda Division B Play Offs
14) Segunda Division B Losers Play Offs
15) Segunda RFEF - Play Out
16) Tercera RFEF - Group 1
17) Tercera RFEF - Group 2
18) Tercera RFEF - Group 3
19) Tercera RFEF - Group 4
20) Tercera RFEF - Group 5
21) Tercera RFEF - Group 6
22) Tercera RFEF - Group 7
23) Tercera RFEF - Group 8
24) Tercera RFEF - Group 9
25) Tercera RFEF - Group 10
26) Tercera RFEF - Group 11
27) Tercera RFEF - Group 12
28) Tercera RFEF - Group 13
29) Tercera RFEF - Group 14
30) Tercera RFEF - Group 15
31) Tercera RFEF - Group 16
32) Tercera RFEF - Group 17
33) Tercera RFEF - Group 18
34) Tercera Division - Play Offs
35) Tercera RFEF - Promot

In [5]:
os.chdir(r"C:\Users\maharvey2101\Documents\Python Scripts\BWFFlashscoreExport")
filenameintro = 'Flashsocre Data up to '
datasheetname = CompetionName + ' Data '

todaydate = datetime.today().strftime('%Y-%m-%d %H.%M.%S')
filename = filenameintro + todaydate +'.xlsx'
writer = pd.ExcelWriter(filename, engine='xlsxwriter')
df.to_excel(writer, sheet_name=datasheetname, index=False)
writer.save()

AttributeError: module 'datetime' has no attribute '__version__'